In [ ]:
# Step 1: Data Preparation (Assuming you have a CSV file 'titanic.csv')
import pandas as pd

data = pd.read_csv('titanic.csv')

# Step 2: Select Relevant Variables
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']
target = 'Survived'

# Step 3: Split the Data
from sklearn.model_selection import train_test_split

train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)
dev_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Step 4: Data Preprocessing
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
for feature in features:
    if data[feature].dtype == 'object':
        train_data[feature] = encoder.fit_transform(train_data[feature])
        dev_data[feature] = encoder.transform(dev_data[feature])
        test_data[feature] = encoder.transform(test_data[feature])

X_train = train_data[features]
y_train = train_data[target]
X_dev = dev_data[features]
y_dev = dev_data[target]

# Step 5: Create Models and Tune Parameters
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

models = [
    ("Bagged", BaggingClassifier(n_estimators=100, random_state=42), {}),
    ("Random Forest", RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42), {}),
    ("Boosted", GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42), {})
]

best_model = None
best_accuracy = 0

for model_name, model, params in models:
    model.fit(X_train, y_train)
    y_dev_pred = model.predict(X_dev)
    dev_accuracy = accuracy_score(y_dev, y_dev_pred)
    
    if dev_accuracy > best_accuracy:
        best_accuracy = dev_accuracy
        best_model = (model_name, model, params)

# Step 6: Report Model Accuracies and Best Model
print("Model Accuracies:")
for model_name, model, params in models:
    model.fit(X_train, y_train)
    y_dev_pred = model.predict(X_dev)
    dev_accuracy = accuracy_score(y_dev, y_dev_pred)
    print(f"{model_name}: {dev_accuracy:.2f}")

best_model_name, best_model, best_params = best_model
print(f"\nBest Model: {best_model_name}")
print(f"Best Dev Accuracy: {best_accuracy:.2f}")
print(f"Best Model Parameters: {best_params}")

# Step 7: Report Test Accuracy of Best Model
X_test = test_data[features]
y_test = test_data[target]
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy of Best Model: {test_accuracy:.2f}")
